In [3]:
import time
import torch
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import WordNetLemmatizer
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
#from transformers import GPT2Tokenizer, GPT2Config, GPT2ForSequenceClassification, Trainer, TrainingArguments
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Happy
[nltk_data]     Iguare\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import time
import pandas as pd
import numpy as np
import nltk
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.cuda.amp import GradScaler, autocast

nltk.download('wordnet')


In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, Dataset, RandomSampler, TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [7]:
test_data = pd.read_csv('test.csv', encoding='ISO-8859-1')
train_data = pd.read_csv('train.csv', encoding='ISO-8859-1')

print(test_data.head())
# Print the original lengths
print("Original train data length:", len(train_data))
print("Original test data length:", len(test_data))

# Drop rows with missing 'text'
train_data.dropna(subset=['text'], inplace=True)
test_data.dropna(subset=['text'], inplace=True)

# Count 'neutral' sentiments before filtering them out
neutral_train_count = train_data[train_data['sentiment'] == 'neutral'].shape[0]
neutral_test_count = test_data[test_data['sentiment'] == 'neutral'].shape[0]

print("Number of neutral sentiments in training data:", neutral_train_count)
print("Number of neutral sentiments in testing data:", neutral_test_count)

# Define sentiment mapping
sentiment_mapping = {
    'negative': 0,
    'positive': 1,
    'neutral': 2,
}

# Apply the mapping to the sentiment column in both DataFrames
train_data['sentimentLabel'] = train_data['sentiment'].map(sentiment_mapping)
test_data['sentimentLabel'] = test_data['sentiment'].map(sentiment_mapping)

# Drop any rows that failed to map
train_data.dropna(subset=['sentimentLabel'], inplace=True)
test_data.dropna(subset=['sentimentLabel'], inplace=True)

# Filter out the 'neutral' sentiments
train_data = train_data[train_data['sentimentLabel'] != 2]
test_data = test_data[test_data['sentimentLabel'] != 2]

# Simplify the datasets to contain only 'text' and 'encoded sentiment'
train = train_data[['text', 'sentimentLabel']]
test = test_data[['text', 'sentimentLabel']]

# Print the updated lengths
print("Updated train data length:", len(train))
print("Updated test data length:", len(test))

# Separate features and labels
X_train = train['text']
y_train = train['sentimentLabel']
X_test = test['text']
y_test = test['sentimentLabel']


       textID                                               text sentiment  \
0  f87dea47db  Last session of the day  http://twitpic.com/67ezh   neutral   
1  96d74cb729   Shanghai is also really exciting (precisely -...  positive   
2  eee518ae67  Recession hit Veronique Branquinho, she has to...  negative   
3  01082688c6                                        happy bday!  positive   
4  33987a8ee5             http://twitpic.com/4w75p - I like it!!  positive   

  Time of Tweet Age of User      Country  Population -2020  Land Area (Km²)  \
0       morning        0-20  Afghanistan        38928346.0         652860.0   
1          noon       21-30      Albania         2877797.0          27400.0   
2         night       31-45      Algeria        43851044.0        2381740.0   
3       morning       46-60      Andorra           77265.0            470.0   
4          noon       60-70       Angola        32866272.0        1246700.0   

   Density (P/Km²)  
0             60.0  
1            1

In [20]:
def lemmatize_text(texts):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in text.split()]) for text in texts]

# Enhanced pipeline with VotingClassifier
start_time = time.time()
pipeline = Pipeline([
    ('lemmatize', FunctionTransformer(lambda x: lemmatize_text(x), validate=False)),
    ('tfidf', TfidfVectorizer(lowercase=True, max_features=5000)),
    ('clf', VotingClassifier(estimators=[
        ('svc', SVC(probability=True, kernel='linear', C=1.0)),
        ('mnb', MultinomialNB(alpha=1.0))
    ], voting='soft')),
])

param_grid = {
    'tfidf__ngram_range': [(1, 2)],
    'tfidf__min_df': [5],
    'tfidf__max_df': [0.75],
    'clf__svc__C': [1.0],
    'clf__mnb__alpha': [1.0,10.0],
}

cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search with the updated pipeline and StratifiedKFold
grid_search = GridSearchCV(pipeline, param_grid, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Predictions and classification report
pred = best_model.predict(X_test)
print("Enhanced Classification Report")
print(classification_report(y_test, pred, target_names=['negative', 'positive']))
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")


Enhanced Classification Report
              precision    recall  f1-score   support

    negative       0.87      0.90      0.88      1001
    positive       0.90      0.88      0.89      1103

    accuracy                           0.89      2104
   macro avg       0.89      0.89      0.89      2104
weighted avg       0.89      0.89      0.89      2104

Training completed in 293.77 seconds.


from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
import time

svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', lowercase=True)),  # Use built-in English stop words
    ('svd', TruncatedSVD(n_components=100)),  # Reducing dimensions
    ('clf', SVC(probability=True, class_weight='balanced')),
])

svm_param_grid = {
    'tfidf__ngram_range': [(1, 2)],
    'tfidf__min_df': [3],
    'tfidf__max_df': [0.75],
    'svd__n_components': [100],
    'clf__C': [0.1, 1],
    'clf__kernel': ['linear'],
    'clf__gamma': ['scale'],
}

kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

start_time = time.time()

svm_grid_search = GridSearchCV(svm_pipeline, svm_param_grid, cv=kfolds, scoring='accuracy', n_jobs=-1, verbose=3)
svm_grid_search.fit(X_train, y_train)

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

svm_best_model = svm_grid_search.best_estimator_
svm_pred = svm_best_model.predict(X_test)

print("Optimized SVM Classification Report:")
print(classification_report(y_test, svm_pred, target_names=['negative', 'positive']))


**bert**

In [10]:
def encode_reviews(tokenizer, reviews, max_length=64):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = encode_reviews(tokenizer, X_train)
test_encodings = encode_reviews(tokenizer, X_test)

train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 3  
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

scaler = GradScaler()

start_time = time.time()
model.train()
for epoch in range(3):  
    for batch in train_loader:
        batch = tuple(t.to(model.device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        optimizer.zero_grad()
        with autocast():
            outputs = model(**inputs)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(model.device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]
        }
        outputs = model(**inputs)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).tolist())
        true_labels.extend(batch[2].tolist())

end_time = time.time()
total_time = end_time - start_time
accuracy = accuracy_score(true_labels, predictions)
print(f"Total execution time: {total_time:.2f} seconds")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\ProgramData\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.14021682739257812
Epoch 2, Loss: 0.07911890000104904
Epoch 3, Loss: 0.00796283408999443
Total execution time: 1063.79 seconds


In [11]:
bert_report = classification_report(true_labels, predictions, target_names=['negative', 'positive'], digits=4)
print("Classification Report:\n", bert_report)

Classification Report:
               precision    recall  f1-score   support

    negative     0.9401    0.9570    0.9485      1001
    positive     0.9604    0.9447    0.9525      1103

    accuracy                         0.9506      2104
   macro avg     0.9503    0.9509    0.9505      2104
weighted avg     0.9507    0.9506    0.9506      2104



**gpt2**

In [18]:
def encode_reviews(tokenizer, reviews, max_length=64):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")

# Initialize GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  

train_encodings = encode_reviews(tokenizer, X_train)
test_encodings = encode_reviews(tokenizer, X_test)

train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)
model.config.pad_token_id = tokenizer.eos_token_id
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 3 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


scaler = GradScaler()

start_time = time.time()
model.train()
for epoch in range(3):  
    for batch in train_loader:
        batch = tuple(t.to(model.device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        optimizer.zero_grad()
        with autocast():
            outputs = model(**inputs)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

model.eval()
gpt_predictions, gpt_true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(model.device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]
        }
        outputs = model(**inputs)
        logits = outputs.logits
        gpt_predictions.extend(torch.argmax(logits, dim=-1).tolist())
        gpt_true_labels.extend(batch[2].tolist())

end_time = time.time()
total_time = end_time - start_time
accuracy = accuracy_score(gpt_true_labels, gpt_predictions)
print(f"Total execution time: {total_time:.2f} seconds")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\ProgramData\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.16836963593959808
Epoch 2, Loss: 0.512739360332489
Epoch 3, Loss: 0.20878435671329498
Total execution time: 5201.76 seconds


In [19]:
gpt_report = classification_report(gpt_true_labels, gpt_predictions, target_names=['negative', 'positive'], digits=4)
print("Classification Report:\n", gpt_report)

Classification Report:
               precision    recall  f1-score   support

    negative     0.9289    0.9391    0.9339      1001
    positive     0.9441    0.9347    0.9394      1103

    accuracy                         0.9368      2104
   macro avg     0.9365    0.9369    0.9367      2104
weighted avg     0.9369    0.9368    0.9368      2104

